In [11]:
import numpy as np
import numba #Librería que permite acelerar funciones de python mediante compilación just-in-time (JIT)

In [12]:
@numba.jit(nopython=True)
def _tridiagonal_solve(a,b,c,d):
    n=len(d)
    cp=np.empty(n-1)
    dp=np.empty(n)
    cp[0]=c[0]/b[0]
    dp[0]=d[0]/b[0]
    for i in range(1,n-1):
        denom=b[i]-a[i-1]*cp[i-1]
        cp[i]=c[i]/denom
        dp[i]=(d[i]-a[i-1]*dp[i-1])/denom
    dp[n-1]=(d[n-1]-a[n-2]*dp[n-2])/(b[n-1]-a[n-2]*cp[n-2])
    x=np.empty(n)
    x[n-1]=dp[n-1]
    for i in range(n-2,-1,-1):
        x[i]=dp[i]-cp[i]*x[i+1]
    return x

def _domain_from_L(K,sigma,T,L):
    xmin=np.log(K)-L*sigma*np.sqrt(T)
    xmax=np.log(K)+L*sigma*np.sqrt(T)
    return xmin,xmax

def _bc_arrays_full(is_call,american,K,r,q,Smin,Smax,tau_vec):
    if is_call:
        left=np.zeros_like(tau_vec)
        right=np.exp(-q*tau_vec)*Smax-K*np.exp(-r*tau_vec)
    else:
        left=K*np.ones_like(tau_vec) if american else K*np.exp(-r*tau_vec)
        right=np.zeros_like(tau_vec)
    return left,right

def solve_pde(K,T,r,sigma,q=0.0,is_call=False,american=True,M=200,N=1000,L=6.0,xmin=None,xmax=None,return_grid=False,method="implicit"):
    if xmin is None or xmax is None:
        xmin,xmax=_domain_from_L(K,sigma,T,L)
    x=np.linspace(xmin,xmax,M+1)
    S=np.exp(x)
    dx=(xmax-xmin)/M
    dt=T/N
    a=0.5*sigma*sigma
    b=r-q-0.5*sigma*sigma
    alpha=a*dt/(dx*dx)
    beta=b*dt/(2.0*dx)
    gamma=r*dt
    U=np.empty((N+1,M+1))
    payoff=np.maximum(S-K,0.0) if is_call else np.maximum(K-S,0.0)
    U[0]=payoff
    tau_vec=np.arange(N+1)*dt
    Smin=np.exp(xmin)
    Smax=np.exp(xmax)
    left_bc,right_bc=_bc_arrays_full(is_call,american,K,r,q,Smin,Smax,tau_vec)
    U[:,0]=left_bc
    U[:,-1]=right_bc
    if method=="explicit":
        a_star=( -beta + alpha )/(1.0+gamma)
        b_star=( 1.0 - 2.0*alpha )/(1.0+gamma)
        g_star=( beta + alpha )/(1.0+gamma)
        for n in range(N):
            U[n+1,1:-1]=a_star*U[n,0:-2]+b_star*U[n,1:-1]+g_star*U[n,2:]
            if american:
                U[n+1,1:-1]=np.maximum(U[n+1,1:-1],payoff[1:-1])
    elif method=="implicit":
        lower=(beta - alpha)*np.ones(M-1)
        diag=(1.0 + 2.0*alpha + gamma)*np.ones(M-1)
        upper=(-(beta + alpha))*np.ones(M-1)
        for n in range(N):
            rhs=U[n,1:-1].copy()
            rhs[0]-= lower[0]*U[n+1,0]
            rhs[-1]-= upper[-1]*U[n+1,-1]
            U[n+1,1:-1]=_tridiagonal_solve(lower,diag,upper,rhs)
            if american:
                U[n+1,1:-1]=np.maximum(U[n+1,1:-1],payoff[1:-1])
    else:
        raise ValueError("method must be 'explicit' or 'implicit' per Hull's scheme")
    if return_grid:
        return S,U[-1],U
    return S,U[-1]